In [1]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [2]:
def preprocess_text(text):
    text=text.lower()
    words=word_tokenize(text)
    stop_word=set(stopwords.words('english'))
    words=[word for word in words if word not in stop_word]
    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in words]

    proprocessed_text=' '.join(words)
    return proprocessed_text

In [3]:
import pandas as pd
try:
    ds=pd.read_csv('question_set.csv',encoding='ISO-8859-1')
    print(ds.head(2))
except UnicodeDecodeError as err:
    print(f"Error: {err}")

   sr_no                                           question  \
0    1.0  What is Python? List some popular applications...   
1    2.0  What are the benefits of using Python language...   

                                                 ans  \
0  Python is a widely-used general-purpose, high-...   
1  Object-Oriented Language\nHigh-Level Language\...   

                                                ans1  
0  Python is a widely-used general-purpose, high-...  
1  Object-Oriented Language\nHigh-Level Language\...  


In [4]:
ds.isnull().sum()

sr_no       1
question    1
ans         1
ans1        1
dtype: int64

In [5]:
ds.fillna("Not Mentioned",inplace=True)


C:\Users\hp\AppData\Local\Temp\ipykernel_24892\1406131915.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Mentioned' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ds.fillna("Not Mentioned",inplace=True)


In [6]:
ds['ans1']=ds['ans1'].apply(preprocess_text)
ds.head(1)

,sr_no,question,ans,ans1
0,1.0,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...","python widely-us general-purpos , high-level p..."


In [7]:
import pickle
with open('preprocess_data.pkl','wb')as file:
    pickle.dump(ds,file)

In [8]:
ds['question']=ds['question'].str.strip()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')
    

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
def suggest_sections(ans,ds,min_suggestions=1):
    preprocessed_ans=preprocess_text(ans)
    ans_embedding=model.encode(preprocessed_ans)
    section_embedding=model.encode(ds['ans1'].tolist())
    similarities=util.pytorch_cos_sim(ans_embedding,section_embedding)[0]
    similarity_threhold=0.2
    relevant_indices=[]
    while len(relevant_indices)<min_suggestions and similarity_threhold>0:
        relevant_indices=[i for i, sim in enumerate(similarities)if sim>similarity_threhold]
        similarity_threhold-=0.5 #st=st-0.5
        sorted_indices=sorted(relevant_indices,key=lambda i: similarities[i],reverse=True)
        suggestions=[
            {
                'index':i,
                'question':ds.iloc[i]['question'],
                'ans':ds.iloc[i]['ans'],
                'similarity_score':similarities[i].item()

            }
            for i in sorted_indices
        ]
    return suggestions

In [11]:
print("Following are the questions")
print(ds['question'])
n=int(input("Which question you want to ans"))
ans=input("Enter you ans")

suggestions=suggest_sections(ans,ds,min_suggestions=1)

suggestion=suggestions[n-1]

if suggestion:
        print(f"sr_no: {suggestion['index']+1}")
        print(f"question: {suggestion['question']}")
        print(f"Your ans :{ans}")
        print(f"Expected ans: {suggestion['ans']}")
        print(f"similarity score : {round(suggestion['similarity_score']*100,2)}%")
        print("_________________________________________________________________________________________\n")
else:
    print("No record is found")


Following are the questions
0    What is Python? List some popular applications...
1    What are the benefits of using Python language...
2    What is the difference between a Mutable datat...
3    What is the difference between a Set and Dicti...
4                            What is a pass in Python?
5    Difference between for loop and while loop in ...
6                                        Not Mentioned
Name: question, dtype: object


IndexError: list index out of range